In [1]:
from controller.blockchain import Blockchain
from controller import magic_key

In [2]:
magic_key.generate_magic_keys("proof_mk.bin", "priv_key_proof.pem", "pub_key_proof.pem")

In [3]:
magic_key = magic_key.get_magic_key("proof_mk.bin", "priv_key_proof.pem")

In [4]:
print(magic_key)

b'g9\xc4%\xdb\xea\xe9B7\xac\xb9\x05\r\xb6;u'


In [5]:
blockchain = Blockchain(magic_key, file_path="Proof_of_ownership.docx", chunks=10)

In [6]:
splitted_data = blockchain._chunk_data()

In [7]:
for index, bin_data in enumerate(splitted_data):
    data = blockchain.encrypt_block(bin_data)
    previous_block = blockchain.get_previous_block()
    previous_nonce = previous_block['nonce']
    nonce, current_hash = blockchain.nonce_and_hash(previous_nonce)
    previous_hash = blockchain.hash(previous_block)
    block = blockchain.create_block(data, nonce, current_hash, previous_hash, title=blockchain.file)

In [8]:
from pprint import pprint
pprint(blockchain.chain)

[{'block_title': 'Proof_of_ownership.docx_0001',
  'data': 'Blockchain created by: Geoffroy Givry',
  'hash': '00000000000000000',
  'index': 1,
  'nonce': 1,
  'previous_hash': '0',
  'timestamp': '2018-09-21T16:04:34.209230'},
 {'block_title': 'Proof_of_ownership.docx_0002',
  'data': {'ciphertext': '...',
           'header': 'UHJvb2Zfb2Zfb3duZXJzaGlwLmRvY3ggc2VudCBieSBHZW9mZnJveSBHaXZyeS4=',
           'nonce': 'Mn3+59fnQmDLU7gzaBU0+g==',
           'tag': 'DP2aKeQ+yn0SsvPXxqcAhQ=='},
  'hash': '0000c00870f23a23ae80377298491b091db400d575be0efbde5b310f2f763ed1',
  'index': 2,
  'nonce': 533,
  'previous_hash': '04bf38031458e20b9f4cc74e2ed972c83d1e50cfbd3ed859a0bb88b763b458b1',
  'timestamp': '2018-09-21T16:04:34.269035'},
 {'block_title': 'Proof_of_ownership.docx_0003',
  'data': {'ciphertext': '...',
           'header': 'UHJvb2Zfb2Zfb3duZXJzaGlwLmRvY3ggc2VudCBieSBHZW9mZnJveSBHaXZyeS4=',
           'nonce': '+6KbcAblZUCZB2nDJxVpBQ==',
           'tag': 'iPtQWqvSByJti1Km7cmaqg=='},


 {'block_title': 'Proof_of_ownership.docx_0007',
  'data': {'ciphertext': '...',
           'header': 'UHJvb2Zfb2Zfb3duZXJzaGlwLmRvY3ggc2VudCBieSBHZW9mZnJveSBHaXZyeS4=',
           'nonce': 'kKHD4YOTqIAJWtGPKQJLnw==',
           'tag': '84al8+n3GZ05yDHqVT5Jyw=='},
  'hash': '0000da4012ef88a954b83b6d188fca1d354973ded35762e3fabb875024aa2768',
  'index': 7,
  'nonce': 19865,
  'previous_hash': '0d7e2c5e37408b327ca5abd8ad06045453f74b68f77174c3894aff7d671018fa',
  'timestamp': '2018-09-21T16:04:34.791138'},
 {'block_title': 'Proof_of_ownership.docx_0008',
  'data': {'ciphertext': '...',
           'header': 'UHJvb2Zfb2Zfb3duZXJzaGlwLmRvY3ggc2VudCBieSBHZW9mZnJveSBHaXZyeS4=',
           'nonce': 'poUqfu+miSCJe+mhGvie4Q==',
           'tag': 'HzvZEEdH0t4OSYlmFBy77g=='},
  'hash': '0000b03e4be7c262e1ac7cc3259731b98bdaffda79850bbf7cf42b98cdb363f3',
  'index': 8,
  'nonce': 95063,
  'previous_hash': 'fcddc6ded66ed83314e502dbc5647ca2fa32dd66d760bcddf7c2a0fdef0da217',
  'timestamp': '2018-09-21T16:

In [9]:
blockchain.is_chain_valid(blockchain.chain)

True

In [10]:
from pymongo import MongoClient
from atlas.config import glpr_config as glc

server = MongoClient(glc.MONGODB)
db = server.gleipnir

In [11]:
db[blockchain.title].insert_many(blockchain.chain)
